##### Copyright 2019 The TensorFlow Authors.



In [0]:
#@title Licensed under the Apache License, Version 2.0
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Addons Optimizers: LazyAdam

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/addons/tutorials/optimizers_lazyadam"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/addons/blob/master/docs/tutorials/optimizers_lazyadam.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/addons/blob/master/docs/tutorials/optimizers_lazyadam.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

This notebook will demonstrate how to use the lazy adam optimizer from the Addons package.


## LazyAdam

> LazyAdam is a variant of the Adam optimizer that handles sparse updates more efficiently.
    The original Adam algorithm maintains two moving-average accumulators for
    each trainable variable; the accumulators are updated at every step.
    This class provides lazier handling of gradient updates for sparse
    variables.  It only updates moving-average accumulators for sparse variable
    indices that appear in the current batch, rather than updating the
    accumulators for all indices. Compared with the original Adam optimizer,
    it can provide large improvements in model training throughput for some
    applications. However, it provides slightly different semantics than the
    original Adam algorithm, and may lead to different empirical results.

## Setup

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
try:
  %tensorflow_version 2.x
except:
  pass

import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
!pip install --no-deps tensorflow-addons~=0.6

  Using cached https://files.pythonhosted.org/packages/43/b0/6a1dacc2f4fab422926bfcbab6fa8f08f2a0309d872f3b059340a409b194/tensorflow_addons-0.6.0-cp36-cp36m-manylinux2010_x86_64.whl


In [0]:
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import numpy as np
from matplotlib import pyplot as plt

In [0]:
# Hyperparameters
batch_size=64
epochs=10

## Build the Model

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_shape=(784,), activation='relu', name='dense_1'),
    tf.keras.layers.Dense(64, activation='relu', name='dense_2'),
    tf.keras.layers.Dense(10, activation='softmax', name='predictions'),
])

## Prepare the Data

In [0]:
# Load MNIST dataset as NumPy arrays
dataset = {}
num_validation = 10000
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data
x_train = x_train.reshape(-1, 784).astype('float32') / 255
x_test = x_test.reshape(-1, 784).astype('float32') / 255

11493376/11490434 [==============================] - 0s 0us/step


## Train and Evaluate

Simply replace typical keras optimizers with the new tfa optimizer 

In [0]:
# Compile the model
model.compile(
    optimizer=tfa.optimizers.LazyAdam(0.001),  # Utilize TFA optimizer
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

# Train the network
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs)


Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 48us/sample - loss: 0.0900 - accuracy: 0.9728
Epoch 2/10
60000/60000 [==============================] - 2s 40us/sample - loss: 0.0710 - accuracy: 0.9781
Epoch 3/10
60000/60000 [==============================] - 2s 40us/sample - loss: 0.0602 - accuracy: 0.9816
Epoch 4/10
60000/60000 [==============================] - 2s 40us/sample - loss: 0.0500 - accuracy: 0.9845
Epoch 5/10
60000/60000 [==============================] - 2s 40us/sample - loss: 0.0414 - accuracy: 0.9874
Epoch 6/10
60000/60000 [==============================] - 2s 39us/sample - loss: 0.0370 - accuracy: 0.9880
Epoch 7/10
60000/60000 [==============================] - 2s 40us/sample - loss: 0.0329 - accuracy: 0.9893
Epoch 8/10
60000/60000 [==============================] - 2s 40us/sample - loss: 0.0274 - accuracy: 0.9915
Epoch 9/10
60000/60000 [==============================] - 2s 40us/sample - loss: 0.0226 - accuracy: 0.9927
Epoch 10/10
60

In [0]:
# Evaluate the network
print('Evaluate on test data:')
results = model.evaluate(x_test, y_test, batch_size=128, verbose = 2)
print('Test loss = {0}, Test acc: {1}'.format(results[0], results[1]))

Evaluate on test data:
10000/1 - 0s - loss: 0.0458 - accuracy: 0.9761
Test loss = 0.0915067508997512, Test acc: 0.9761000275611877
